In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.callbacks import TensorBoard, ModelCheckpoint

print('Imports done.')


Using TensorFlow backend.


Imports done.


In [2]:
print('Reading csv...')
data = pd.read_csv('../input/train_cleaned.csv')
data = data.dropna()
print('Done...')

Reading csv...
Done...


In [3]:
print('Loading pickles...')
train_question_1 = pickle.load(open('train_question_1.pkl', 'rb'))
train_question_2 = pickle.load(open('train_question_2.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
map_index_vec = pickle.load(open('map_index_vec.pkl', 'rb'))

print('Done.')

Loading pickles...
Done.


In [4]:
dim = 50
N = len(train_question_1)
batch_size = 128
epochs = 100
log_dir = './logs'
weights_dir = './weights_1/weights.hdf5'

In [5]:
len(range(1,N))

404285

In [6]:
train_data = []
print('Creating Training data...');
for i, sentence1, sentence2 in zip(range(N), train_question_1, train_question_2):
    if i % 50000 == 0:
        print(i)
        
    vector1 = [map_index_vec[index] for index in sentence1]
    vector2 = [map_index_vec[index] for index in sentence2]
    mean_vec1 = np.mean(vector1, axis=0)
    mean_vec2 = np.mean(vector2, axis=0)

    train_data.append(np.concatenate((mean_vec1, mean_vec2)))
print('Done.')

Creating Training data...
0
50000
100000
150000
200000
250000
300000
350000
400000
Done.


In [7]:
print('Converting data to array...')
train_data = np.array(train_data)
train_labels = np.array(train_labels)
print('Done.')

Converting data to array...
Done.


In [8]:
print('Building model...')
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_shape=(2 * dim,)))
print(model.input_shape)
print(model.output_shape)
sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

tb = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=False)
checkpointer = ModelCheckpoint(filepath=weights_dir, verbose=1, save_best_only=True)
print('Done.')

Building model...
(None, 100)
(None, 1)
Done.


In [9]:
train_data.shape

(404286, 100)

In [10]:
print('Training...')
model.fit(train_data, train_labels,
          epochs=epochs,
          validation_split=0.2,
          batch_size=batch_size, callbacks=[tb, checkpointer])
print('Done.')

Training...
Train on 323428 samples, validate on 80858 samples
Epoch 1/100
323428/323428 [==============================] - 5s - loss: 0.6471 - acc: 0.6315 - val_loss: 0.6346 - val_acc: 0.6480
Epoch 2/100
323428/323428 [==============================] - 5s - loss: 0.6380 - acc: 0.6400 - val_loss: 0.6316 - val_acc: 0.6518
Epoch 3/100
323428/323428 [==============================] - 5s - loss: 0.6362 - acc: 0.6437 - val_loss: 0.6304 - val_acc: 0.6529
Epoch 4/100
323428/323428 [==============================] - 4s - loss: 0.6355 - acc: 0.6457 - val_loss: 0.6303 - val_acc: 0.6500
Epoch 5/100
323428/323428 [==============================] - 4s - loss: 0.6352 - acc: 0.6463 - val_loss: 0.6295 - val_acc: 0.6552
Epoch 6/100
323428/323428 [==============================] - 3s - loss: 0.6349 - acc: 0.6476 - val_loss: 0.6292 - val_acc: 0.6549
Epoch 7/100
323428/323428 [==============================] - 4s - loss: 0.6347 - acc: 0.6478 - val_loss: 0.6295 - val_acc: 0.6556
Epoch 8/100
323428/323428 [